In [4]:
# 라이브러리 및 모듈 설치
import tempfile
import os
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
from tensorflow import keras
from modul_test_1 import prune_and_train_model, quantize_and_train_model

In [5]:
#데이터 테스트 셋과 저장된 모델 불러오기
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0


model = tf.keras.models.load_model("my_model_test.h5")
_, accuracy = model.evaluate(
   test_images, test_labels, verbose=0)

In [6]:
model_for_pruning, model_for_pruning_accuracy = prune_and_train_model(
    model, train_images, train_labels, test_images, test_labels,
    128, 3, 0.2
)

Epoch 1/3
375/375 [==============================] - 5s 9ms/step - loss: 0.0910 - accuracy: 0.9744 - val_loss: 0.0911 - val_accuracy: 0.9764
Epoch 2/3
375/375 [==============================] - 3s 9ms/step - loss: 0.1041 - accuracy: 0.9720 - val_loss: 0.1049 - val_accuracy: 0.9711
Epoch 3/3
375/375 [==============================] - 3s 9ms/step - loss: 0.0969 - accuracy: 0.9722 - val_loss: 0.0942 - val_accuracy: 0.9740
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshap  (None, 28, 28, 1)         1         
 e (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_conv2d  (None, 26, 26, 12)        230       
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_max_po  

In [7]:
# 가중치 가지치기 기능 적용 모델 저장
model_for_pruning.save("purned_model_test.h5")

c:\Users\Jay\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
q_aware_model, q_aware_model_accuracy = quantize_and_train_model(
    model, train_images, train_labels, test_images, test_labels,
    128, 3, 0.2
)

Epoch 1/3
375/375 [==============================] - 5s 12ms/step - loss: 0.0866 - accuracy: 0.9751 - val_loss: 0.0875 - val_accuracy: 0.9743
Epoch 2/3
375/375 [==============================] - 4s 11ms/step - loss: 0.0754 - accuracy: 0.9777 - val_loss: 0.0828 - val_accuracy: 0.9761
Epoch 3/3
375/375 [==============================] - 4s 11ms/step - loss: 0.0692 - accuracy: 0.9796 - val_loss: 0.0804 - val_accuracy: 0.9764
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_1 (Quantize  (None, 28, 28)            3         
 Layer)                                                          
                                                                 
 quant_reshape (QuantizeWra  (None, 28, 28, 1)         1         
 pperV2)                                                         
                                                                 
 quant_conv2d (QuantizeWra

In [11]:
# 양자화 기능 적용 모델 저장
q_aware_model.save("quantized_model_test.h5")

In [ ]:
# 기능 매트릭 정리 
# 파일 크기를 정수로 변환하여 출력
keras_file_size = int(model_size("my_model_test.h5"))
pruned_keras_file_size = int(model_size("purned_model_test.h5"))
quant_file_size = int(model_size("quantized_model_test.h5"))
# 압축률 = (압축 전 데이터 크기) / (압축 후 데이터 크기)
pruned_keras_compression_rate = ((keras_file_size / pruned_keras_file_size))
quant_file_compression_rate = (keras_file_size / quant_file_size)
pruned_keras_compression_rate = "{:.2f}".format(pruned_keras_compression_rate)
quant_file_compression_rate = "{:.2f}".format(quant_file_compression_rate)
# 원본 모델 비교
PADP = (accuracy - (accuracy - model_for_pruning_accuracy)) / accuracy * 100
if PADP >= 100:
    PADP = 100
P = "{:.2f}".format(PADP)
PADQ = (accuracy - (accuracy - q_aware_model_accuracy)) / accuracy * 100
if PADQ >= 100:
    PADQ = 100
Q = "{:.2f}".format(PADQ)

In [ ]:
# 각 모델간(원본, 가중치 가지치기, 양자화) 성능비교 매트릭 출력
print('모델 사이즈---------------------------------------------------')
print("Size of gzipped baseline Keras model: %d bytes" % keras_file_size)
print("Size of gzipped pruned Keras model: %d bytes" % pruned_keras_file_size)
print("Size of gzipped Quantized model: %d bytes" % quant_file_size)
print('모델 압축률---------------------------------------------------')
print("compression rate of pruned Keras model: ", pruned_keras_compression_rate)
print("compression rate of Quantized model: ", quant_file_compression_rate)
print('모델 최적화 성능----------------------------------------------')
print('Baseline test accuracy:', accuracy)
print('Pruned test accuracy:', model_for_pruning_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)
print('모델 최적화 비교 정확도----------------------------------------')
print('가지치기 비교 정확도 :', P,"%")
print('양자화 비교 정확도 :', Q,"%")